In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
%config IPCompleter.greedy=True

In [2]:
spark = SparkSession.builder.appName("Python Spark").master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

## 1. Read

In [3]:
df=spark.read.format("json").load("mini.json")
df

corporator,dateTS,dayOrderSeq,discount,discountRate,discountType,erase,faceID,isSigned,memberID,moneyBeforeWholeDiscount,operator,operatorName,orderID,origin,payStatus,payType,payedTotal,paymentChannel,paymentScenarios,product,productCount,receivable,serverSaved,smallChange,storeAddress,storeCategory,storeCity,storeCreateDateTS,storeDistrict,storeGPSAddress,storeGPSLatitude,storeGPSLongitude,storeGPSName,storeID,storeName,storeOwnUserId,storeOwnUserName,storeOwnUserTel,storeProvince,storeShopNo,storeStatus,totalNoDiscount
hnzy,1563758583000,8,0.0,1,2,0.0,,0,0,22.5,NameStr,OperatorName,15637585824094064...,0,-1,alipay,22.5,0,PASV,"[{6940159410029, ...",4.0,22.5,true,0.0,StoreAddress,normal,长沙市,1557733046000,雨花区,,28.121213726311993,113.01567856440359,,4064,杨光峰南食店,4082,OwnUserNameStr,12345678910,湖南省,NULL,open,22.5
NULL,1546737450000,6,0.0,1,2,0.0,,1,0,7.0,NameStr,OperatorName,15467374316307186813,0,NULL,alipay,7.0,0,PASV,"[{6901028191012, ...",1.0,7.0,NULL,0.0,StoreAddress,normal,长沙市,1534920455000,岳麓区,None,28.158909,112.95106,None,718,芙蓉兴盛汶强食品店,577,OwnUserNameStr,12345678910,湖南省,277551253310005,open,7.0
NULL,1546478081000,26,0.0,1,2,0.0,,0,0,10.0,NameStr,OperatorName,15464780808601786...,0,NULL,cash,10.0,0,OTHER,"[{6901028000826, ...",1.0,10.0,NULL,0.0,StoreAddress,normal,长沙市,1540802164000,芙蓉区,None,28.198982,113.03312,None,1786,裕丰超市,1714,OwnUserNameStr,12345678910,湖南省,None,open,10.0
hnzy,1559133703000,144,0.0,1,2,0.0,,0,0,10.5,NameStr,OperatorName,15591337021343702...,0,-1,wechat,10.5,0,PASV,"[{6970855690265, ...",2.0,10.5,true,0.0,StoreAddress,normal,深圳市,1552977317000,龙华区,,,,,3702,天福便利店,3692,OwnUserNameStr,12345678910,广东省,NULL,open,10.5
NULL,1548594458000,57,0.0,1,2,0.0,,0,0,10.0,NameStr,OperatorName,15485944578681156...,0,NULL,cash,10.0,0,OTHER,"[{6901028010108, ...",1.0,10.0,NULL,0.0,StoreAddress,normal,桂林市,1537601490000,象山区,None,25.248390759610427,110.28186410665509,None,1156,吉顺食品自选商店,1040,OwnUserNameStr,12345678910,广西壮族自治区,None,open,10.0
NULL,1548292824000,36,0.0,1,2,0.0,,1,0,3.0,NameStr,OperatorName,15482928201153183933,0,NULL,wechat,3.0,1,PURC,"[{4891028164395, ...",1.0,3.0,NULL,0.0,StoreAddress,normal,珠海市,1518114308000,香洲区,None,22.274088374908985,113.51813376850318,None,318,芙蓉兴盛3626,229,OwnUserNameStr,12345678910,广东省,277585253310024,open,3.0
NULL,1545356344000,12,0.0,1,2,0.0,,0,0,6.5,NameStr,OperatorName,15453563437071699...,0,NULL,cash,6.5,0,OTHER,"[{6901028191012, ...",1.0,6.5,NULL,0.0,StoreAddress,normal,长沙市,1540530941000,芙蓉区,None,28.214493749156052,113.01619246602056,None,1699,亿户便利海欣食品店,1620,OwnUserNameStr,12345678910,湖南省,None,open,6.5
NULL,1547284514000,111,0.0,1,2,0.0,,0,0,17.0,NameStr,OperatorName,15472845025471167...,0,NULL,alipay,17.0,0,PASV,"[{99999999-1, 1, ...",2.0,17.0,NULL,0.0,StoreAddress,normal,常德市,1537621831000,武陵区,None,29.05939118356865,111.68978109955786,None,1167,芙蓉兴盛子轩商行,1051,OwnUserNameStr,12345678910,湖南省,None,open,17.0
hnzy,1563845059000,10,0.0,1,2,0.0,,0,0,19.0,NameStr,OperatorName,15638450586263466...,0,-1,cash,19.0,0,OTHER,"[{6932452818091, ...",2.0,19.0,true,0.0,StoreAddress,normal,长沙市,1547892928000,开福区,,28.340324,112.99777,,3466,芙蓉兴盛蔡兰食品店,3434,OwnUserNameStr,12345678910,湖南省,NULL,open,19.0
hft,1557231358000,337,0.0,1,2,0.0,,0,0,4.0,NameStr,OperatorName,15572313547723331851,0,NULL,wechat,4.0,0,PASV,"[{6936455900067, ...",1.0,4.0,true,0.0,StoreAddress,normal,珠海市,1520206281000,香洲区,,22.280635051859548,113.55951672652247,,333,3A福利社,235,OwnUserNameStr,12345678910,广东省,227585253310046,open,4.0


## 2. Clean

In [4]:
df_clean=df.dropna(thresh=1,subset=["storeProvince"]).\
                filter("storeProvince != 'null'").\
                filter("receivable<10000").\
                select("storeProvince","storeID","receivable","dateTS","payType")
df_clean

storeProvince,storeID,receivable,dateTS,payType
湖南省,4064,22.5,1563758583000,alipay
湖南省,718,7.0,1546737450000,alipay
湖南省,1786,10.0,1546478081000,cash
广东省,3702,10.5,1559133703000,wechat
广西壮族自治区,1156,10.0,1548594458000,cash
广东省,318,3.0,1548292824000,wechat
湖南省,1699,6.5,1545356344000,cash
湖南省,1167,17.0,1547284514000,alipay
湖南省,3466,19.0,1563845059000,cash
广东省,333,4.0,1557231358000,wechat


## 3. Data manipulaition

### 3.1  Requirement 1: Sales Revenue Statistics by Province

In [5]:
provicnce_sale=df_clean.groupby("storeProvince").\
    sum("receivable").\
    withColumnRenamed("sum(receivable)", "money").\
    withColumn("money",f.round("money",2)).\
    orderBy("money",ascending=False)
provicnce_sale

storeProvince,money
广东省,1713207.92
湖南省,1701303.53
广西壮族自治区,37828.22
北京市,10926.91
上海市,7358.5
江苏省,6357.9
浙江省,4568.1
山东省,664.0
江西省,553.5


TO MySql

In [6]:
provicnce_sale.write.mode("overwrite").\
    format("jdbc").\
    option("url","jdbc:mysql://host.docker.internal:3306/spark_example?useSSl=false").\
    option("dbtable","province_sale").\
    option("user","root").\
    option("password","123456").\
    option("encoding","utf-8").\
    save()


### 3.2 Among the top 3 sales provinces, how many stores have achieved daily sales of over 1000?

#### get top 3 province

In [7]:
top3_df=provicnce_sale.limit(3).select("storeProvince")
top3_df

storeProvince
广东省
湖南省
广西壮族自治区


#### join  df

In [8]:
top3_joined=df_clean.join(top3_df, on=df_clean["storeProvince"]==top3_df["storeProvince"]).select(df_clean["*"])
top3_joined

storeProvince,storeID,receivable,dateTS,payType
湖南省,4064,22.5,1563758583000,alipay
湖南省,718,7.0,1546737450000,alipay
湖南省,1786,10.0,1546478081000,cash
广东省,3702,10.5,1559133703000,wechat
广西壮族自治区,1156,10.0,1548594458000,cash
广东省,318,3.0,1548292824000,wechat
湖南省,1699,6.5,1545356344000,cash
湖南省,1167,17.0,1547284514000,alipay
湖南省,3466,19.0,1563845059000,cash
广东省,333,4.0,1557231358000,wechat


#### set date formate

In [9]:
top3_joined_date=top3_joined.withColumn("dateTS",f.from_unixtime(top3_joined["dateTS"].substr(0,10),"yyyy-MM-dd"))
top3_joined_date

storeProvince,storeID,receivable,dateTS,payType
湖南省,4064,22.5,2019-07-22,alipay
湖南省,718,7.0,2019-01-06,alipay
湖南省,1786,10.0,2019-01-03,cash
广东省,3702,10.5,2019-05-29,wechat
广西壮族自治区,1156,10.0,2019-01-27,cash
广东省,318,3.0,2019-01-24,wechat
湖南省,1699,6.5,2018-12-21,cash
湖南省,1167,17.0,2019-01-12,alipay
湖南省,3466,19.0,2019-07-23,cash
广东省,333,4.0,2019-05-07,wechat


#### get daily sum

In [10]:
sum_df=top3_joined_date.groupby("storeProvince","storeID","dateTS").sum("receivable").withColumnRenamed("sum(receivable)","day_sale")
sum_df

storeProvince,storeID,dateTS,day_sale
广东省,1286,2019-01-02,4.0
广东省,2520,2019-01-01,25.0
湖南省,2401,2019-05-14,119.0
湖南省,1988,2019-05-28,27.0
湖南省,1644,2018-11-22,96.0
湖南省,1187,2019-01-28,16.0
湖南省,2939,2018-12-12,20.0
湖南省,3352,2019-03-10,15.0
广东省,4019,2019-05-10,6.5
湖南省,1257,2019-01-01,58.0


#### filter

In [11]:
res_df=sum_df.filter("day_sale>=1000").orderBy("storeID",ascending=False)
res_df

storeProvince,storeID,dateTS,day_sale
广东省,4205,2019-07-15,6395.0
广东省,4152,2019-08-10,1939.0
广东省,4111,2019-05-24,1150.0
广东省,4051,2019-06-01,1040.0
广东省,3975,2019-08-23,1050.0
广东省,3903,2019-06-08,1400.0
广东省,3899,2019-04-23,2670.0
广东省,3862,2019-08-01,1035.0
广东省,3848,2019-04-12,3871.0
广东省,3811,2019-05-08,1652.0


#### remove duplicates

In [12]:
remove_df=res_df.dropDuplicates(subset=["storeProvince","storeID"]).orderBy("storeID")
remove_df

storeProvince,storeID,dateTS,day_sale
广东省,277,2019-03-05,1520.0
广东省,301,2019-04-09,1111.0
广东省,355,2019-04-06,1257.0
广东省,376,2019-04-29,1199.0
广东省,384,2018-12-27,2296.0
广东省,389,2019-08-19,1017.0
广东省,407,2019-01-25,4568.5
广东省,520,2019-06-02,1753.0
广东省,562,2019-08-04,1000.0
广东省,571,2019-01-31,3927.0


#### final group

In [13]:
top3_province_with_1000_sale_stores=remove_df.groupby("storeProvince").count()
top3_province_with_1000_sale_stores

storeProvince,count
广西壮族自治区,3
湖南省,108
广东省,108


In [14]:
top3_province_with_1000_sale_stores.write.mode("overwrite").\
    format("jdbc").\
    option("url","jdbc:mysql://host.docker.internal:3306/spark_example?useSSl=false").\
    option("dbtable","top3_province_with_1000_sale_stores").\
    option("user","root").\
    option("password","123456").\
    option("encoding","utf-8").\
    save()

### 3.3 Among the top 3 sales provinces, mean value of  order value

In [20]:
top3_avg_order=top3_joined_date.groupBy("storeProvince")\
    .avg("receivable")\
    .withColumnRenamed("avg(receivable)","avg_order")\
    .withColumn("avg_order",f.round("avg_order",2))\
    .orderBy("avg_order")
top3_avg_order

storeProvince,avg_order
广东省,32.81
湖南省,36.86
广西壮族自治区,40.03


### 3.4 Among the top 3 sales provinces, payment method ratio

In [25]:
select_df=top3_joined_date.select("storeProvince","payType")
select_df

storeProvince,payType
湖南省,alipay
湖南省,alipay
湖南省,cash
广东省,wechat
广西壮族自治区,cash
广东省,wechat
湖南省,cash
湖南省,alipay
湖南省,cash
广东省,wechat


In [27]:
select_df.createOrReplaceTempView("table")

In [34]:
def udf(num):
    return  str(round(num,2)%100)+"%"

In [35]:
spark.udf.register("percent",udf)

<function __main__.udf(num)>

In [38]:
fina_df=spark.sql('''
    select storeProvince,payType , percent(count(1)/total) as ratio
    from (select *, count(1) over(partition by storeProvince) as total from table) as sub
    group by storeProvince, payType , total
''')
fina_df

storeProvince,payType,ratio
广东省,wechat,0.39%
广东省,cash,0.53%
广东省,bankcard,0.01%
广东省,alipay,0.07%
广西壮族自治区,cash,0.73%
广西壮族自治区,wechat,0.22%
广西壮族自治区,alipay,0.04%
广西壮族自治区,bankcard,0.01%
湖南省,alipay,0.04%
湖南省,cash,0.71%
